In [1]:
import gradio as gr
from openai import OpenAI
import json



In [2]:
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [6]:
tools = [{"type" : "function" ,"function" : price_function}]

In [7]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(f"Tool call: {tool_call.function.name} with arguments {tool_call.function.arguments}")
    arguments = json.loads(tool_call.function.arguments)
    print(f"Tool call arguments: {arguments}")
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="llama3.1", messages=messages, tools=tools)
    print(response)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print(messages)
        response = openai.chat.completions.create(model="llama3.1", messages=messages)
    final_text = response.choices[0].message.content
    # if "<think>" in final_text:
    #     final_text = final_text.split("</think>")[-1].strip()
    return  final_text





# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
#     response = openai.chat.completions.create(
#         model="llama3.1",
#         messages=messages
#     )
    
#     final_text = response.choices[0].message.content
    
#     # Remove <think> ... </think> part if present
#     if "<think>" in final_text:
#         final_text = final_text.split("</think>")[-1].strip()
    
#     return final_text

In [ ]:
gr.ChatInterface(fn=chat , type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-366', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_lafd60rl', function=Function(arguments='{"destination_city":"None"}', name='get_ticket_price'), type='function', index=0)]))], created=1755500468, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=19, prompt_tokens=241, total_tokens=260, completion_tokens_details=None, prompt_tokens_details=None))
Tool call: get_ticket_price with arguments {"destination_city":"None"}
Tool call arguments: {'destination_city': 'None'}
Tool get_ticket_price called for None
[{'role': 'system', 'content': "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If y